In [1]:
import os
import re
import json
import time
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [16]:
def get_query(city_name):
    query = str(city_name.encode('utf-8'))[1:].replace("'", "").replace("\\x", "%25").upper()
    return query

def get_sight_url(response):
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'lxml')
    tags = soup.find_all(attrs={"data-beacon": "SC_resault"})
    sight_url = ""
    get_url = False
    if len(tags) != 0:
        sight_url = tags[0]["href"] + "-jingdian"
        get_url = True
    return (sight_url, get_url)

def get_proxy():
    return requests.get("http://127.0.0.1:5010/get/").json()

def delete_proxy(proxy):
    requests.get("http://127.0.0.1:5010/delete/?proxy={}".format(proxy))

def getResponse(base_url, headers):
    # ....
    retry_count = 5
    proxy = get_proxy().get("proxy")
    while retry_count > 0:
        try:
            html = requests.get(base_url, headers=headers, proxies={"http": "http://{}".format(proxy)})
            # 使用代理访问
            return html
        except Exception:
            retry_count -= 1
    # 删除代理池中代理
    delete_proxy(proxy)
    return None
    
data_folder = r"./original_data"
data_file_names = os.listdir(data_folder)
city_names = map(lambda x: x.split('.')[0], data_file_names)
base_url = r"https://travel.qunar.com/search/all/"
params = {}
headers = {}
with open("./output/sight_url.txt", 'a', encoding="utf-8") as f:
    for city_name in city_names:
        print(city_name)
        ua = UserAgent()
        headers["User-Agent"] = ua.random
        url = base_url + get_query(city_name)
        response = getResponse(url, headers)
        print(response.status_code)
        (sight_url, get_url) = get_sight_url(response)
        f.write(city_name)
        f.write("\n")
        f.write(sight_url)
        f.write("\n")
        print(sight_url)
        time.sleep(1)

In [63]:
# 提取代理API接口，获取1个代理IP
api_url = "http://v2.api.juliangip.com/dynamic/getips?auto_white=1&num=1&pt=1&result_type=text&split=1&trade_no=1129780214579896&sign=5e8e521b200e03ade682e8906972d00e"

# 获取API接口返回的代理IP
proxy_ip = requests.get(api_url).text

# 用户名密码认证(动态代理/独享代理)
username = "13437953591"
password = "4XrQ6jfh"
proxies = {
"http": "http://%(user)s:%(pwd)s@%(proxy)s/" % {"user": username, "pwd": password, "proxy": proxy_ip},
"https": "http://%(user)s:%(pwd)s@%(proxy)s/" % {"user": username, "pwd": password, "proxy": proxy_ip},
}

# 白名单方式（需提前设置白名单）
# proxies = {
#     "http": "http://%(proxy)s/" % {"proxy": proxy_ip},
#     "https": "http://%(proxy)s/" % {"proxy": proxy_ip},
# }

# 要访问的目标网页
target_url = "https://www.juliangip.com/api/general/Test"

# 使用代理IP发送请求
response = requests.get(target_url, proxies=proxies)

# 获取页面内容
if response.status_code == 200:
    print(response.text)

{"ip":"110.82.6.157","state":"ok"}


In [ ]:
"<div class="b_paging"><em class="page cur">1</em><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-2">2</a><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-3">3</a><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-4">4</a><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-5">5</a><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-6">6</a><em class="dot">...</em><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-199">199</a><a class="page" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-200">200</a><a class="page next" href="https://travel.qunar.com/p-cs298037-alashanmeng-jingdian-1-2">下一页</a></div>"

# 高铁数据爬取

## 获取所有高铁页面链接

In [407]:
url = "https://train.hao86.com/%E9%AB%98%E9%80%9F/"
headers = {}
ua = UserAgent()
headers["User-Agent"] = ua.random
response = requests.get(url, headers=headers)
html_doc = response.text
soup = BeautifulSoup(html_doc, 'lxml')
uls = soup.find_all("ul", attrs={"id": "scrollbar"})
lis = uls[0].find_all('li')
hsr_data = {
    "hsrs": []
}
for li in lis:
    hsr = {}
    hsr_name = li.contents[0]["href"].replace("/", "")
    hsr["name"]= hsr_name
    hsr["url"] = "https://train.hao86.com" + li.contents[0]["href"]
    hsr_data["hsrs"].append(hsr)
with open("./output/hsr_data.json",'w+') as f:
    json.dump(hsr_data, f)

## 爬取每个页面的内容

In [417]:
def get_stations_table(response):
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'lxml')
    divs = soup.find_all("div", attrs={"class": "main_conter_Article clearfix"})
    get_succ = False
    stations_table = None
    pattern = re.compile(r"途经车站")
    if len(divs) != 0:
        for div in divs:
            for i in range(len(div.contents)):
                child = div.contents[i]
                if child !="\n" and len(child.find_all(string=pattern)) > 0:
                    get_succ = True
                    stations_table = div.find_all("div", attrs={"class": "station"})[0].contents
                    stations_table = list(filter(lambda x: x!="\n", stations_table))[0]
                    break
    return (get_succ, stations_table)

def get_stations(stations_table):
    trs = stations_table.find_all("tr")[1:]
    stations = []
    for tr in trs:
        station = tr.find_all("td")[1].a.string.strip()
        stations.append(station)
    return stations

with open("./output/hsr_data.json",'r+') as f:
    hsr_data = json.load(f)
hsrs = hsr_data["hsrs"]
failed_count = 0
headers = {}
for hsr in hsrs:
    url = hsr["url"]
    ua = UserAgent()
    headers["User-Agent"] = ua.random
    response = getResponse(url, headers=headers)
    if response.status_code != 200:
        print("请求失败！")
    else:
        response.encoding = "utf-8"
        (get_succ, stations_table) = get_stations_table(response)
        if get_succ:
            stations = get_stations(stations_table)
            hsr["stations"] = stations
        else:
            failed_count += 1
            print(f"未找到{hsr["name"]}的途经车站。")
    time.sleep(1)

with open("./output/hsr_data.json",'w+') as f:
    json.dump(hsr_data, f)

未找到G185的途经车站。
未找到G1813的途经车站。
未找到G1822的途经车站。
未找到G1826的途经车站。
未找到G1828的途经车站。
未找到G1832的途经车站。
未找到G1852的途经车站。
未找到G1862的途经车站。
未找到G1866的途经车站。
未找到G1869的途经车站。
未找到G1870的途经车站。
未找到G1874的途经车站。
未找到G1880的途经车站。
未找到G1887的途经车站。
未找到G1896的途经车站。
未找到G1865的途经车站。
未找到G701的途经车站。
未找到G702的途经车站。
未找到G703的途经车站。


AttributeError: 'NoneType' object has no attribute 'string'

## 爬取两个城市之间的最快高铁速度

In [22]:
def time_select(response, start_city, end_city):
    response.encoding = "utf-8"
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'lxml')
    pattern = re.compile(r"^G")
    shortest_time = -1
    tables = soup.find_all("table")
    if len(tables) == 1:
        table = tables[0]
        trs = table.find_all("tr")
        for tr in trs:
            tds = tr.find_all("td")
            if len(tds) > 0:
                hsr_name = tds[0].a.find_all(string=pattern)
                if len(hsr_name) > 0:
                    print(hsr_name[0])
                    time = tr.find_all("td")[-1].string
                    print(time)
    elif len(tables) < 1:
        print(start_city, end_city)
        print("未找到表格！")
    else:
        print("表格数量超过1！")
    return shortest_time


def get_shortest_time(start_city, end_city, headers):
    base_url = "https://train.hao86.com/"
    query = str((start_city + "-" + end_city).encode("utf-8"))[1:].replace("'", "").replace("\\x", "%").upper() + "/"
    url = base_url + query
    response = getResponse(url, headers=headers)
    get_succ = False
    shortest_time = -1
    if response.status_code != 200:
        print("无直达线路或请求失败！")
    else:
        get_succ = True
        shortest_time = time_select(response, start_city, end_city)
    #解析
    return (get_succ, shortest_time)
    

def save_time(start_city, end_city, get_succ, shortest_time, routes_data):
    if get_succ:
        route_name = f"{start_city}-{end_city}"
        routes_time = {
            route_name: shortest_time
        }
        routes_data["routes_time"].append(routes_time)


data_folder = r"./original_data"
data_file_names = os.listdir(data_folder)
city_names = list(map(lambda x: x.split('.')[0], data_file_names))
city_names = city_names[0:10]
count = 0
headers = {}
routes_data = {
    "routes_time": []
}
for i in range(len(city_names)):
    for j in range(i+1, len(city_names)):
        start_city = city_names[i]
        end_city = city_names[j]
        ua = UserAgent()
        headers["User-Agent"] = ua.random
        (get_succ, shortest_time) = get_shortest_time(start_city, end_city, headers)
        save_time(start_city, end_city, get_succ, shortest_time, routes_data)
        (get_succ, shortest_time) = get_shortest_time(end_city, start_city, headers)
        save_time(end_city, start_city, get_succ, shortest_time, routes_data)
        time.sleep(1)

无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
万宁 三亚
未找到表格！
三亚 万宁
未找到表格！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
万宁 东方
未找到表格！
东方 万宁
未找到表格！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
三明 上海
未找到表格！
上海 三明
未找到表格！
G2380
01:30
G322
01:39
G322
01:23
G1654
01:49
G1654
01:33
G3142
01:53
G3142
01:37
G1651
01:19
G1509
01:33
G3141
01:36
G3141
01:53
G2379
01:26
G323
01:23
G323
01:39
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
无直达线路或请求失败！
三门峡 上海
未找到表格！
上海 三门峡
未找到表格！
无直达线路或请求失败！
无直达线路或请求失败

In [18]:
routes_data

{'routes_time': []}